In [1]:
import random


import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

import matplotlib.pyplot as plt
import plotly.express as px

from timeit import default_timer as timer

In [2]:
news_df = pd.read_csv('news.tsv', sep='\t')

behaviors_df = pd.read_csv('behaviors.tsv', sep='\t')


news_df

,N55528,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, and Prince Philip Swear By","Shop the notebooks, jackets, and more that the royals can't live without.",https://assets.msn.com/labs/mind/AAGH0ET.html,"[{""Label"": ""Prince Philip, Duke of Edinburgh"", ""Type"": ""P"", ""WikidataId"": ""Q80976"", ""Confidence"": 1.0, ""OccurrenceOffsets"": [48], ""SurfaceForms"": [""Prince Philip""]}, {""Label"": ""Charles, Prince of Wales"", ""Type"": ""P"", ""WikidataId"": ""Q43274"", ""Confidence"": 1.0, ""OccurrenceOffsets"": [28], ""SurfaceForms"": [""Prince Charles""]}, {""Label"": ""Elizabeth II"", ""Type"": ""P"", ""WikidataId"": ""Q9682"", ""Confidence"": 0.97, ""OccurrenceOffsets"": [11], ""SurfaceForms"": [""Queen Elizabeth""]}]",[]
0,N19639,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...,https://assets.msn.com/labs/mind/AAB19MK.html,"[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik..."
1,N61837,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://assets.msn.com/labs/mind/AAJgNsz.html,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."
2,N53526,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi...",https://assets.msn.com/labs/mind/AACk2N6.html,[],"[{""Label"": ""National Basketball Association"", ..."
3,N38324,health,medical,"How to Get Rid of Skin Tags, According to a De...","They seem harmless, but there's a very good re...",https://assets.msn.com/labs/mind/AAAKEkt.html,"[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI...","[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI..."
4,N2073,sports,football_nfl,Should NFL be able to fine players for critici...,Several fines came down against NFL players fo...,https://assets.msn.com/labs/mind/AAJ4lap.html,"[{""Label"": ""National Football League"", ""Type"":...","[{""Label"": ""National Football League"", ""Type"":..."
...,...,...,...,...,...,...,...,...
51276,N16909,weather,weathertopstories,"Adapting, Learning And Soul Searching: Reflect...",Woolsey Fire Anniversary: A community is forev...,https://assets.msn.com/labs/mind/BBWzQJK.html,"[{""Label"": ""Woolsey Fire"", ""Type"": ""N"", ""Wikid...","[{""Label"": ""Woolsey Fire"", ""Type"": ""N"", ""Wikid..."
51277,N47585,lifestyle,lifestylefamily,Family says 13-year-old Broadway star died fro...,NaN,https://assets.msn.com/labs/mind/BBWzQYV.html,"[{""Label"": ""Broadway theatre"", ""Type"": ""F"", ""W...",[]
51278,N7482,sports,more_sports,St. Dominic soccer player tries to kick cancer...,"Sometimes, what happens on the sidelines can b...",https://assets.msn.com/labs/mind/BBWzQnK.html,[],[]
51279,N34418,sports,soccer_epl,How the Sounders won MLS Cup,"Mark, Jeremiah and Casey were so excited they ...",https://assets.msn.com/labs/mind/BBWzQuK.html,"[{""Label"": ""MLS Cup"", ""Type"": ""U"", ""WikidataId...",[]


In [3]:
print(news_df.columns)

Index(['N55528', 'lifestyle', 'lifestyleroyals',
       'The Brands Queen Elizabeth, Prince Charles, and Prince Philip Swear By',
       'Shop the notebooks, jackets, and more that the royals can't live without.',
       'https://assets.msn.com/labs/mind/AAGH0ET.html',
       '[{"Label": "Prince Philip, Duke of Edinburgh", "Type": "P", "WikidataId": "Q80976", "Confidence": 1.0, "OccurrenceOffsets": [48], "SurfaceForms": ["Prince Philip"]}, {"Label": "Charles, Prince of Wales", "Type": "P", "WikidataId": "Q43274", "Confidence": 1.0, "OccurrenceOffsets": [28], "SurfaceForms": ["Prince Charles"]}, {"Label": "Elizabeth II", "Type": "P", "WikidataId": "Q9682", "Confidence": 0.97, "OccurrenceOffsets": [11], "SurfaceForms": ["Queen Elizabeth"]}]',
       '[]'],
      dtype='object')


In [4]:
news_df.columns = ['article_id', 'category', 'subcategory', 'title', 'abstract', 'url', 'entities', 'extra_data']


In [5]:
news_df

,article_id,category,subcategory,title,abstract,url,entities,extra_data
0,N19639,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...,https://assets.msn.com/labs/mind/AAB19MK.html,"[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik..."
1,N61837,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://assets.msn.com/labs/mind/AAJgNsz.html,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."
2,N53526,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi...",https://assets.msn.com/labs/mind/AACk2N6.html,[],"[{""Label"": ""National Basketball Association"", ..."
3,N38324,health,medical,"How to Get Rid of Skin Tags, According to a De...","They seem harmless, but there's a very good re...",https://assets.msn.com/labs/mind/AAAKEkt.html,"[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI...","[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI..."
4,N2073,sports,football_nfl,Should NFL be able to fine players for critici...,Several fines came down against NFL players fo...,https://assets.msn.com/labs/mind/AAJ4lap.html,"[{""Label"": ""National Football League"", ""Type"":...","[{""Label"": ""National Football League"", ""Type"":..."
...,...,...,...,...,...,...,...,...
51276,N16909,weather,weathertopstories,"Adapting, Learning And Soul Searching: Reflect...",Woolsey Fire Anniversary: A community is forev...,https://assets.msn.com/labs/mind/BBWzQJK.html,"[{""Label"": ""Woolsey Fire"", ""Type"": ""N"", ""Wikid...","[{""Label"": ""Woolsey Fire"", ""Type"": ""N"", ""Wikid..."
51277,N47585,lifestyle,lifestylefamily,Family says 13-year-old Broadway star died fro...,NaN,https://assets.msn.com/labs/mind/BBWzQYV.html,"[{""Label"": ""Broadway theatre"", ""Type"": ""F"", ""W...",[]
51278,N7482,sports,more_sports,St. Dominic soccer player tries to kick cancer...,"Sometimes, what happens on the sidelines can b...",https://assets.msn.com/labs/mind/BBWzQnK.html,[],[]
51279,N34418,sports,soccer_epl,How the Sounders won MLS Cup,"Mark, Jeremiah and Casey were so excited they ...",https://assets.msn.com/labs/mind/BBWzQuK.html,"[{""Label"": ""MLS Cup"", ""Type"": ""U"", ""WikidataId...",[]


In [6]:
len(news_df)

51281

In [7]:
dropped_df = news_df.sample(frac=0.99, random_state=42)
dropped_article_ids = dropped_df['article_id'].tolist()
train_news_reduced = news_df.drop(dropped_df.index)

print(len(train_news_reduced))

513


In [8]:
train_news_reduced

,article_id,category,subcategory,title,abstract,url,entities,extra_data
161,N16430,video,animals,This Rare Albino Hedgehog Was Rescued In Calif...,This hedgehog was rescued in California.,https://assets.msn.com/labs/mind/AAHZ3Ph.html,"[{""Label"": ""California"", ""Type"": ""G"", ""Wikidat...","[{""Label"": ""California"", ""Type"": ""G"", ""Wikidat..."
189,N24900,sports,mma,Bare Knuckle Knockout! Watch Gabriel Gonzaga W...,Bare Knuckle FC returned on Saturday night for...,https://assets.msn.com/labs/mind/AAJ44h4.html,"[{""Label"": ""Ant\u00f4nio Silva (fighter)"", ""Ty...","[{""Label"": ""Ant\u00f4nio Silva (fighter)"", ""Ty..."
190,N9947,finance,finance-career-education,10 Job Skills Worth Six-Figure Salaries,Learn these skills to get you closer to six fi...,https://assets.msn.com/labs/mind/AAEzJ4H.html,[],[]
197,N15651,finance,finance-real-estate,The penthouse of NYC's Woolworth Building just...,"The condo comes as a ""white-box"" unit, meaning...",https://assets.msn.com/labs/mind/AAJJaaX.html,"[{""Label"": ""Woolworth Building"", ""Type"": ""F"", ...",[]
206,N37423,weather,weathertopstories,"Suez says lead levels meet federal standards, ...",Suez said Friday lead levels now meets the fed...,https://assets.msn.com/labs/mind/AAEzQxF.html,[],"[{""Label"": ""Test cricket"", ""Type"": ""U"", ""Wikid..."
...,...,...,...,...,...,...,...,...
50680,N10213,weather,weathertopstories,Cold front could plunge Houston area into wint...,"It's only November, but freezing temperatures ...",https://assets.msn.com/labs/mind/BBWyj9D.html,"[{""Label"": ""Greater Houston"", ""Type"": ""G"", ""Wi...","[{""Label"": ""Greater Houston"", ""Type"": ""G"", ""Wi..."
50800,N3042,travel,travelarticle,Tug boat captain remembers day Edmund Fitzgera...,"On Nov. 10, 1975, the SS Edmund Fitzgerald san...",https://assets.msn.com/labs/mind/BBWymXD.html,"[{""Label"": ""SS Edmund Fitzgerald"", ""Type"": ""V""...","[{""Label"": ""SS Edmund Fitzgerald"", ""Type"": ""V""..."
50845,N35652,travel,travelnews,"Connelly: The ongoing, back door bid to commer...",Our national park system was and is America's ...,https://assets.msn.com/labs/mind/BBWyoCt.html,[],"[{""Label"": ""Outdoor recreation"", ""Type"": ""C"", ..."
50859,N25890,sports,football_nfl,Minneapolis church votes to remove controversi...,Members of Plymouth Congregational Church in M...,https://assets.msn.com/labs/mind/BBWyog9.html,"[{""Label"": ""Minneapolis"", ""Type"": ""G"", ""Wikida...","[{""Label"": ""Minneapolis"", ""Type"": ""G"", ""Wikida..."


In [9]:
# Nettoyage simple sur les titres des articles
train_news_reduced['title'] = train_news_reduced['title'].str.lower().str.replace('[^\w\s]', '')

# Gestion des données manquantes (par exemple, remplir avec une valeur par défaut)
train_news_reduced.fillna('Unknown', inplace=True)


In [10]:
category_counts = train_news_reduced['category'].value_counts()
px.bar(category_counts, title="Nombre d'articles par catégorie")

In [11]:
train_news_reduced['article_id']

161      N16430
189      N24900
190       N9947
197      N15651
206      N37423
          ...  
50680    N10213
50800     N3042
50845    N35652
50859    N25890
51005    N40220
Name: article_id, Length: 513, dtype: object

In [12]:
vectorizer = TfidfVectorizer(max_features=35000, stop_words='english')
tfidf_matrix = vectorizer.fit_transform(train_news_reduced['abstract'])
cosine_sim = cosine_similarity(tfidf_matrix)
cosine_df = pd.DataFrame(cosine_sim, index=train_news_reduced['article_id'], columns=train_news_reduced['article_id'])


In [13]:
cosine_df

article_id,N16430,N24900,N9947,N15651,N37423,N7362,N61456,N44396,N60825,N13833,...,N24680,N62502,N40993,N393,N5903,N10213,N3042,N35652,N25890,N40220
article_id,,,,,,,,,,,,,,,,,,,,,
N16430,1.0,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
N24900,0.0,1.000000,0.0,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.024901,0.000000,0.054901,0.000000,0.000000,0.0
N9947,0.0,0.000000,1.0,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
N15651,0.0,0.000000,0.0,1.000000,0.00000,0.0,0.000000,0.061248,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.052889,0.000000,0.0
N37423,0.0,0.000000,0.0,0.000000,1.00000,0.0,0.000000,0.000000,0.024422,0.000000,...,0.032948,0.017618,0.0,0.000000,0.000000,0.000000,0.008710,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
N10213,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.048635,0.000000,0.000000,0.000000,...,0.000000,0.038599,0.0,0.018295,0.000000,1.000000,0.028791,0.028014,0.000000,0.0
N3042,0.0,0.054901,0.0,0.000000,0.00871,0.0,0.000000,0.000000,0.013747,0.000000,...,0.009552,0.029528,0.0,0.000000,0.000000,0.028791,1.000000,0.000000,0.018282,0.0
N35652,0.0,0.000000,0.0,0.052889,0.00000,0.0,0.000000,0.000000,0.000000,0.027211,...,0.000000,0.000000,0.0,0.035447,0.000000,0.028014,0.000000,1.000000,0.000000,0.0


In [14]:
tfidf_matrix

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 9103 stored elements and shape (513, 4611)>

In [15]:
tfidf_matrix

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 9103 stored elements and shape (513, 4611)>

In [16]:
from profiles import create_profiles
from srPredicteur import pred_article
from llmPredicteur import pred_article_llm

# Metrics

In [17]:
def diversity_intra_list(recommended_items, cosine_df, train_news_reduced):
    """
    Calcul de la diversité intra-liste en fonction de la similarité des items.

    recommended_items: Liste des items recommandés.
    cosine_df: DataFrame ou matrice contenant les similarités entre les articles.
    train_news_reduced: DataFrame contenant les articles avec les indices correspondants.
    """
    # Calcul de la diversité intra-liste (moyenne des distances)
    diversity_score = 0
    n = len(recommended_items)

    if n <= 1 :
        return 0

    for i in range(n):
        for j in range(i + 1, n):
            # Récupérer l'index des articles dans train_news_reduced
            article_id_i = recommended_items[i]
            article_id_j = recommended_items[j]

            # Utiliser les indices dans cosine_df pour calculer la similarité
            # Assurez-vous que l'index de cosine_df correspond à l'index de train_news_reduced
            sim = cosine_df.loc[article_id_i, article_id_j]  # Similarité entre les deux articles
            diversity_score += (1 - sim)  # 1 - similarité = distance

    # Normalisation pour obtenir un score entre 0 et 1
    return diversity_score / (n * (n - 1) / 2)  # Normalisation


In [18]:
threshold = 0.005
binary_similarity_matrix = (cosine_df >= threshold).astype(int)

num_articles = len(cosine_df)
mean_connection_all = np.mean(binary_similarity_matrix)
print(mean_connection_all)

0.12947953596358233


In [19]:
random.seed(42)

In [20]:
U_id, U_liked, U_watched = create_profiles(num_users=2)
n_iter = 3

diversity_score = pd.DataFrame({"iteration" : range(1, n_iter + 1)})
timer_score = pd.DataFrame({"iteration" : range(1, n_iter + 1)})
connection_score = pd.DataFrame({"iteration" : range(1, n_iter + 1)})

In [21]:
# Simulation de n_iter, suppose que vous calculez le score de diversité pour un algorithme à chaque itération
diversity_per_iter=[]
connection_per_iter=[]
end_t = []
for _ in range(n_iter):
    mean_diversity = 0
    mean_connection = 0
    start_t = timer()
    user_liked, user_watched, all_news = pred_article(U_id, U_liked, U_watched, train_news_reduced, cosine_df)
    end_t.append(timer() - start_t)

    # Calculer le score de diversité moyen pour tous les utilisateurs à cette itération
    for user in U_id.index:
        # diversity score
        recommended_news = all_news[all_news.user_id == user]["recommended_news"].values[0]  # Liste des articles recommandés
        mean_diversity += diversity_intra_list(recommended_news, cosine_df, train_news_reduced)

        # connectivity score
        l = len(all_news[all_news.user_id == user]["recommended_news"].values[0])
        if l != 0 :
            ms = 0
            for new in all_news[all_news.user_id == user]["recommended_news"].values[0]:
                s = (np.sum(binary_similarity_matrix[new]) -1 )
                ms += s / l

        mean_connection += ms


    mean_diversity /= len(U_id.index)
    mean_connection /= len(U_id.index)

    # Ajouter le score de diversité pour cette itération
    diversity_per_iter.append(mean_diversity)
    connection_per_iter.append(mean_connection)

    print(f"Mean Diversity for iteration {_ + 1}: {mean_diversity}")

# Créer un DataFrame avec le score de diversité pour chaque itération
print(len(diversity_score))
print(len(diversity_per_iter))
diversity_score['SR'] =  diversity_per_iter
connection_score['SR'] = connection_per_iter
timer_score['SR'] = end_t

# Affichage du DataFrame pour vérifier
print(diversity_score)



Mean Diversity for iteration 1: 0.9988781826021732
Mean Diversity for iteration 2: 0.9913314594182228
Mean Diversity for iteration 3: 0.9925485891228139
3
3
   iteration        SR
0          1  0.998878
1          2  0.991331
2          3  0.992549


In [ ]:
LLMs_algo = ['mistral']
for llm in LLMs_algo :
    # Simulation de n_iter, suppose que vous calculez le score de diversité pour un algorithme à chaque itération
    diversity_per_iter=[]
    connection_per_iter=[]
    end_t = []
    for _ in range(n_iter):
        mean_diversity = 0
        mean_connection = 0
        start_t = timer()
        user_liked, user_watched, all_news = pred_article_llm(U_id, U_liked, U_watched, train_news_reduced, model=llm)
        end_t.append(timer() - start_t)

        # Calculer le score de diversité moyen pour tous les utilisateurs à cette itération
        for user in U_id.index:
            # diversity score
            recommended_news = all_news[all_news.user_id == user]["recommended_news"].values[0]  # Liste des articles recommandés
            mean_diversity += diversity_intra_list(recommended_news, cosine_df, train_news_reduced)

            # connectivity score
            l = len(all_news[all_news.user_id == user]["recommended_news"].values[0])
            if l != 0 :
                ms = 0
                for new in all_news[all_news.user_id == user]["recommended_news"].values[0]:
                    s = (np.sum(binary_similarity_matrix[new]) -1 )
                    ms += s / l

            mean_connection += ms


        mean_diversity /= len(U_id.index)
        mean_connection /= len(U_id.index)

        # Ajouter le score de diversité pour cette itération
        diversity_per_iter.append(mean_diversity)
        connection_per_iter.append(mean_connection)

        print(f"Mean Diversity for iteration {_ + 1}: {mean_diversity}")

    # Créer un DataFrame avec le score de diversité pour chaque itération
    print(len(diversity_score))
    print(len(diversity_per_iter))
    diversity_score[llm] =  diversity_per_iter
    connection_score[llm] = connection_per_iter
    timer_score[llm] = end_t

    # Affichage du DataFrame pour vérifier
    print(diversity_score)



Mean Diversity for iteration 1: 0.9983748403913846


## Results

In [ ]:
# Création du graphique avec Plotly
fig = px.line(diversity_score, x='iteration', y=['SR', 'mistral'], labels={'iteration': 'Iteration', 'SR': 'Diversity Score'},
              title='Diversity Score for SR Algorithm Over Iterations')

# Affichage du graphique
fig.show()

In [ ]:
# Création du graphique avec Plotly
fig = px.line(timer_score, x='iteration', y=['SR', 'mistral'], labels={'iteration': 'Iteration', 'SR': 'Timer Score'},
              title='Time Score for SR Algorithm Over Iterations')

# Affichage du graphique
fig.show()

In [ ]:
# Création du graphique avec Plotly
fig = px.line(connection_score, x='iteration', y=['SR', 'mistral'], labels={'iteration': 'Iteration', 'SR': 'Connection Score'},
              title='Connectivity Score for SR Algorithm Over Iterations')

# Affichage du graphique
fig.show()